# Вебинар 4. Домашнее задание

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from metrics import precision_at_k, recall_at_k
from utils import prefilter_items

In [2]:
data = pd.read_csv('./lessons/web2/retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)

test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
item_features = pd.read_csv('./lessons/web2/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [5]:
n_items_before = data_train['item_id'].nunique()

interesting_cats = ['PASTRY', 'PRODUCE', 'DRUG GM', 'FLORAL']
data_train = prefilter_items(
    data_train,
    item_features,
    take_n_popular=5000,
    interesting_cats=interesting_cats
)

n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 86865 to 23544


In [6]:
user_item_matrix = pd.pivot_table(
    data_train, 
    index='user_id',
    columns='item_id', 
    values='quantity',
    aggfunc='count', 
    fill_value=0
)
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

user_item_matrix.head(3)

item_id,26601,26889,27334,28165,28219,28346,28424,29421,29618,29751,...,13775932,13776505,13776773,13776774,13777140,13777389,13777425,13876695,13877031,13910384
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [8]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [9]:
%%time

model = AlternatingLeastSquares(
    factors=20, 
    regularization=0.001,
    iterations=15, 
    calculate_training_loss=True, 
    num_threads=4
)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

100%|██████████| 15/15 [00:26<00:00,  1.77s/it, loss=0.0098]Wall time: 26.7 s



-----

# Домашнее задание

1. Перенесите метрики в модуль metrics.py (убедится что они там)
2. Перенесите функцию prefilter_items в модуль utils.py
3. Создайте модуль recommenders.py. Напищите код для класса ниже 
(задание обсуждали на вебинаре, для первой функции практически сделали) и положите его в recommenders.py
4. Проверьте, что все модули корректно импортируются

In [10]:
from metrics import precision_at_k, recall_at_k
from utils import prefilter_items
from recommenders import MainRecommender

recom = MainRecommender(
    data=data_train,
    weighting_type=None,
)

100%|██████████| 23544/23544 [00:00<00:00, 67078.50it/s]


## Категории товаров, купленных юзером user_id=1

In [11]:
user_id = 1

## Рекомендация топ-5 похожих товаров

In [12]:
items_rec = recom.get_similar_items_recommendation(user_id, N=5)
items_rec

[867405, 1055646, 9189814, 863762, 852486]

In [13]:
item_features[item_features['item_id'].isin(items_rec)]

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
9704,852486,1808,PRODUCE,National,BROCCOLI/CAULIFLOWER,BROCCOLI WHOLE&CROWNS,
10983,863762,6046,DRUG GM,National,PREPAID WIRELESS&ACCESSORIES,PREPAID WIRELESS CARDS,
11392,867405,261,DRUG GM,National,CANDY - PACKAGED,CANDY BAGS-NON CHOCOLATE,10.5 OZ
32587,1055646,2,PRODUCE,National,VEGETABLES SALAD,HEAD LETTUCE,24CT
64681,9189814,1660,DRUG GM,National,MAGAZINE,SOCIAL&LIFESTYLE-MAGAZINE,


In [14]:
# result = data_test.groupby('user_id')['item_id'].unique().reset_index()
# result.columns=['user_id', 'actual']

# print(f'precision_at_k: {precision_at_k(items_rec, result[result["user_id"] == 1]["actual"])}')
# print(f'recall_at_k: {recall_at_k(items_rec, result[result["user_id"] == 1]["actual"])}')

## Рекомендация топ-5 товаров похожих пользователей

In [15]:
items_user_rec = recom.get_similar_users_recommendation(user_id, N=5)
items_user_rec

[866211, 8020166, 1075368, 1082185, 904360]

In [16]:
item_features[item_features['item_id'].isin(items_user_rec)]

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
11258,866211,2,PRODUCE,National,GRAPES,GRAPES WHITE,18 LB
15504,904360,673,PRODUCE,National,VEGETABLES SALAD,HEAD LETTUCE,24CT
34770,1075368,539,DRUG GM,National,CIGARETTES,CIGARETTES,974246 PK
35576,1082185,2,PRODUCE,National,TROPICAL FRUIT,BANANAS,40 LB
61964,8020166,1225,DRUG GM,National,INFANT FORMULA,INFANT FORMULA SOY BASE,12.9 OZ
